In [3]:
import os

import psycopg2
from dotenv import load_dotenv


load_dotenv()

True

In [4]:
db_params = {
    "host": "127.0.0.1",
    "port": os.environ["POSTGRES_PORT"],
    "database": os.environ["POSTGRES_DB"],
    "user": os.environ["POSTGRES_USER"],
    "password": os.environ["POSTGRES_PASSWORD"],
}

In [6]:
from smolagents import tool


@tool
def sql_engine(query: str) -> str:
    """
    Execute an arbitrary SQL statement against the **resumes** table and return
    the result as a string.

    Args:
        query (str): A valid PostgreSQL statement (SELECT, INSERT, UPDATE or
            DELETE).  
            • For SELECT queries, add `LIMIT 100` (recommended) to avoid huge  
              result sets.  
            • Quote column names with double quotes if they contain uppercase
              letters or non-ASCII symbols.

    Returns:
        str:  
            • If the statement yields rows (e.g. SELECT), a stringified Python
              list of tuples, e.g.  
              ``"[ (1, 'Зыкова Валерия Кузьминична', 'женский', ...), ... ]"``  
            • If no rows are returned (e.g. UPDATE), the literal message  
              ``"Query executed successfully, but no results to fetch."``

    Table schema (resumes):
        id              INTEGER  - primary key
        name            TEXT     - full name (UTF-8)
        gender          TEXT     - «женский», «мужской», etc.
        occupation      TEXT     - current/target role (nullable)
        summary         TEXT     - short profile with description  of position
        contact_info    JSONB    - {"email": "...", "phone": "...", ...}
        experience      JSONB    - list of work-experience blocks
        education       JSONB    - list of education blocks
        portfolio       JSONB    - list of projects {"name", "link", "description"}
        languages       TEXT[]   - ['Русский – родной', 'Английский – B2', ...]
        skills          TEXT[]   - ['Kubernetes', 'Python', ...]
        certifications  TEXT[]   - certificates
        hobbies         TEXT[]   - hobbies

    Examples:
        >>> sql_engine(\"""
            SELECT id, name, occupation
            FROM resumes
            WHERE 'Kubernetes' = ANY(skills)
            ORDER BY id
            LIMIT 5;
        \""")
        "[ (4, 'Маргарита Кирилловна Дорофеева', 'DevOps Engineer'), ... ]"

        >>> sql_engine(\"""
            UPDATE resumes
            SET occupation = 'Mobile Developer'
            WHERE id = 1;
        \""")
        "Query executed successfully, but no results to fetch."

    Important:
        • Never pass raw user input directly into *query*; always parameterize
          to avoid SQL injection.  
        • Avoid requesting more than 1 000 rows per call to keep responses
          manageable for the language model.
    """
    output: Union[List[Tuple[Any, ...]], str]

    conn: psycopg2.extensions.connection | None = None
    try:
        conn = psycopg2.connect(**db_params)
        with conn.cursor() as cursor:
            cursor.execute(query)

            try:
                rows: List[Tuple[Any, ...]] = cursor.fetchall()
                output = rows
            except psycopg2.ProgrammingError:
                output = "Query executed successfully, but no results to fetch."

        conn.commit()
    finally:
        if conn is not None:
            conn.close()

    return str(output)



In [7]:
from smolagents import tool


@tool
def sql_engine(query: str) -> str:
    """Allow to perform SQL queries on the table. Returns a string representation of the result.

        The table is named 'resumes'. Its description is as follows:
            - resumes Table Schema for 'resumes':
            - id (integer)
            - contact_info (jsonb)
            - experience (jsonb)
            - education (jsonb)
            - portfolio (jsonb)
            - languages (ARRAY)
            - skills (ARRAY)
            - certifications (ARRAY)
            - hobbies (ARRAY)
            - name (text)
            - gender (text)
            - title (text)
            - summary (text) First 5 rows from 'resumes':



        First row from 'resumes':
            id | name | gender | title | summary | contact_info | skills | experience | education | languages | certifications | hobbies | portfolio
            1 | Зыкова Валерия Кузьминична | женский | Mobile Developer | Опытный мобильный разработчик с 5-летним стажем в разработке и оптимизации мобильных приложений. Обладаю глубоким пониманием современных технологий и платформ, таких как iOS и Android. Сильные навыки в проектировании, разработке и тестировании приложений, а также в работе в команде и управлении проектами. Ищу возможность применить свои навыки и знания в динамичной и инновационной компании. | {'email': 'valeriya.zykova@example.com', 'phone': '+7 (808) 262-35-84', 'github': 'github.com/valeriya-zykova', 'linkedin': 'linkedin.com/in/valeriya-zykova', 'location': 'Чехия'} | ['Swift', 'Kotlin', 'React Native', 'Firebase', 'Git', 'Agile/Scrum', 'UX/UI Design', 'RESTful APIs', 'CI/CD', 'Test-Driven Development'] | [{'company': 'TechSolutions', 'end_date': '2023-05-31', 'job_title': 'Mobile Developer', 'start_date': '2018-06-01', 'achievements': ['Разработала и запустила 3 мобильных приложения для iOS и Android, которые достигли более 100 000 загрузок.', 'Оптимизировала производительность приложений, сократив время загрузки на 30%.', 'Внедрила CI/CD пайплайны, что сократило время развертывания на 50%.', 'РаЬотала в мультидисциплинарной команде, координируя усилия разработчиков, дизайнеров и тестировщиков.']}] | [{'degree': 'Бакалавр информационных технологий', 'details': 'Специализация: Программная инженерия', 'end_date': '2017-06-30', 'start_date': '2013-09-01', 'institution': 'Санкт-Петербургский Политехнический Университет'}] | ['Русский – родной', 'Английский – B2', 'Чешский – A2'] | ['Google Certified Professional Cloud Developer', 'Apple Developer Program'] | ['Фотография', 'Путешествия', 'Программирование в свободное время'] | [{'link': 'github.com/valeriya-zykova/TravelApp', 'name': 'TravelApp', 'description': 'Мобильное приложение для планирования путешествий. Использовались технологии: Swift, Firebase, MapKit. Приложение позволяет пользователям создавать маршруты, добавлять точки интереса и делиться планами с друзьями.'}, {'link': 'github.com/valeriya-zykova/FitnessTracker', 'name': 'FitnessTracker', 'description': 'Приложение для отслеживания физической активности. Использовались технологии: Kotlin, Google Fit API, Room Database. Приложение позволяет пользователям отслеживать свои тренировки, устанавливать цели и получать уведомления о прогрессе.'}]

        Args:
            query: The query to perform. This should be correct SQL.

    """
    output = ""
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute(query)

    try:
        output = cursor.fetchall()
    except psycopg2.ProgrammingError:
        output = "Query executed successfully, but no results to fetch."

    conn.commit()
    conn.close()
    return output

In [9]:
from smolagents import CodeAgent, OpenAIServerModel, InferenceClientModel, ToolCallingAgent
from smolagents import LiteLLMModel


model = OpenAIServerModel(
    model_id="small",
    api_key=os.environ["AGENT_LLM_API_TOKEN"],
    api_base=os.environ["AGENT_LLM_API_URL"],
    flatten_messages_as_text=True,
)



agent = CodeAgent(
    tools=[sql_engine],
    model=model,
    planning_interval=5,
    description = 'Ты - агент HR, который ищет в базе данных информацию о сотрудниках. Предобработай овтет пользователя, составь план и выполни',
    max_steps = 5,
)
agent.run("сколько людей девопс")

KeyError: 'AGENT_LLM_API_URL'